In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 61 kB 383 kB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#collapse-hide
INSTALL_MSG = """
Bart will be released through pip in v 3.0.0, until then use it by installing from source:

git clone git@github.com:huggingface/transformers.git
git checkout d6de6423
cd transformers
pip install -e ".[dev]"

"""

import torch
try:
    import transformers
    from transformers import BartTokenizer, BartForConditionalGeneration
except ImportError:
    raise ImportError(INSTALL_MSG)
from IPython.display import display, Markdown

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

covid = """Of the 18 patients on this Covid ward in Leipzig, 14 were unvaccinated
One of Germany's top virologists has warned that a further 100,000 people will die from Covid if nothing's done to halt an aggressive fourth wave.Ê
Case numbers have soared and Germany on Wednesday registered its highest rate of infection since the pandemic began, with almost 40,000 cases in a day.Ê
"We have to act right now," said Christian Drosten, who described a real emergency situation.
Doctors in the intensive care Covid ward at Leipzig University Hospital warn this fourth wave could be the worst yet.Ê
One patient here, a woman in her 20s, has just given birth. Her baby is fine, but staff say they don't know whether she'll survive.Ê
This state of Saxony has the highest seven-day infection rate in Germany at 459 cases per 100,000 people. The national rate is 232.
It also has the lowest take-up of vaccine: 57% of the population here have been vaccinated.Ê
There are 18 patients on the Covid ward. Just four were vaccinated.Ê
"It's very difficult to get staff motivated to treat patients now in this fourth wave," says Prof Sebastian Stehr, who heads the department. "A large part of the population still underestimates the problem."
By now, Prof Sehr says, most people will know someone who's had Covid and should, therefore, be aware of the risk of infection. "Nevertheless," he adds, "we are still seeing so many patients who are not vaccinated."
Germany's health minister has publicly blamed those people for the soaring cases, describing the current situation as a "pandemic of the unvaccinated".
At the start of this week, Saxony banned unvaccinated people from bars, restaurants, public events and sport and leisure facilities. At least several other states are expected to follow suit.Ê
Germany's anti-vaxxers are furious. Several thousand protested last weekend in Leipzig.Ê
"This is discrimination, and we want to express vehemently that we do not accept this in our society," said Leif Hansen, who represents anti-vax "Bewegung Leipzig" (Leipzig Movement). He doesn't trust the companies that made the vaccine or the authorities who approved it.Ê
"They say the vaccination is ok, that I should give it to my children? Never!" he told me. "I have a feeling that it should never go into my body, and I will fight all I can to prevent it coming into my body."
Anti-vaxxers out in force in the city of Leipzig, in Saxony: This eastern state has the lowest vaccine take-up rate in Germany
Sixteen million Germans over the age of 12 have not been fully vaccinated. The German government has admitted it's unlikely to persuade many of those people now, and politicians worry that social divisions might deepen.Ê
What many fear is another lockdown. Nadine Herzog's bar in Leipzig barely survived the last one. She deftly prepares cocktails as she tells us that even before the authorities required it she and her co-owner banned unvaccinated drinkers.Ê
"My business is dying. My dreams came true and now they suffer from people who don't do logical things to prevent others from getting ill or dying and I'm so angry," she told the BBC.
A long queue formed outside a vaccine centre in the city earlier in the week. Evidence, perhaps, that some have changed their minds. But Germany is also now rolling out booster jabs amid concerns about waning protection in those who have been vaccinated.Ê
On the intensive care Covid unit they fear the damage is done. Operations have already been cancelled, procedures postponed to make way for Covid patients.Ê
Doctors here told us that nearly half of the people who end up here will die.Ê
For Germany, which invented one of the world's first Covid vaccines, that is a source of great shame.
""".replace('\n','')

In [ ]:
#collapse-show
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


# ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
#>>> inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')


#>>> summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
#>>> print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])
article_input_ids = tokenizer.batch_encode_plus([covid], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=142,
                             min_length=56,
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


> **Summary: **Of the 18 patients on this Covid ward in Leipzig, 14 were unvaccinated. Saxony has the highest seven-day infection rate in Germany at 459 cases per 100,000 people. It also has the lowest take-up of vaccine: 57% of the population here have been vaccinated.

In [ ]:
summary_ids.shape

torch.Size([1, 66])